In [1]:
!conda install -c anaconda beautifulsoup4 --yes
!conda install -c anaconda html5lib --yes
!conda install -c anaconda lxml --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.0       |           py36_0         165 KB  anaconda
    ca-certificates-2020.1.1   |                0         132 KB  anaconda
    certifi-2020.4.5.1         |           py36_0         159 KB  anaconda
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    ------------------------------------------------------------
                                           Total:         4.2 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1      --> 4.9.0-py36_0      anaconda
    ca-certificates: 2020.1.1-0        --> 2020.1.1-0        anaconda
    certifi:         2020.4.5.1-py36_0 --> 2020.4.5.1-py36_0 anaconda
    openssl:         1.1.1g-

## 1. scrape the data from Wikipedia page and load into a dataframe

In [14]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = BeautifulSoup(source,'lxml')

table = soup.find_all('table')
df = pd.read_html(str(table))[0]

df.head(5)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [15]:
df.shape

(180, 3)

## 2. Ignore cells with a borough that is Not assigned.

In [16]:
Toronto_df = df[df['Borough'] != 'Not assigned']
Toronto_df.shape

(103, 3)

In [17]:
Toronto_df.dtypes

Postal Code     object
Borough         object
Neighborhood    object
dtype: object

## 3. Aggregrate neighborhood for each postal code area

In [19]:
Toronto_DataSet = pd.pivot_table(Toronto_df, index=['Postal Code','Borough'], 
                        values=['Neighborhood'], 
                        aggfunc=lambda x: ', '.join(map(str, x)))

In [20]:
Toronto_DataSet.dtypes

Neighborhood    object
dtype: object

In [21]:
Toronto_DataSet.shape

(103, 1)

In [22]:
Toronto_DataSet.reset_index(inplace=True)
Toronto_DataSet.head(5)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## 4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [23]:
Filter = (Toronto_DataSet['Neighborhood'] == 'Not assigned') & (Toronto_DataSet['Borough'].notnull())
Toronto_DataSet.loc[Filter, 'Neighborhood'] = Toronto_DataSet.loc[Filter, 'Borough']

In [24]:
Toronto_DataSet.shape

(103, 3)